#### Edit Data Paths
##### Edit input and outputs paths. Input path should direct to the csv that has the sequences to create the embeddings for, and the output path should direct to the csv that will hold the embeddings.

In [1]:
!pwd

/home/romi/projects/preTCR/lab_notebooks


In [2]:
# the sequences in the csv file need to be in a column called Sequences
input_path = "/home/romi/projects/preTCR/CDR3_data/cdr3_reg_labeled.csv"
output_path = "/home/romi/projects/preTCR/CDR3_data/cdr3_reg_embeddings.csv"

#### Set Environment

In [3]:
cd ..

/home/romi/projects/preTCR


In [4]:
import os, sys
import pandas as pd
SRC_DIR = "tcr"
assert os.path.isdir(SRC_DIR), f"Cannot find src dir: {SRC_DIR}"
sys.path.append(SRC_DIR)

from tcr import model_utils

from lab_notebooks.utils import TRANSFORMER, DEVICE
MODEL_DIR = os.path.join(SRC_DIR, "models")
sys.path.append(MODEL_DIR)

FILT_EDIT_DIST = True

#### Load Data

In [ ]:
# load data
tcrb_data = pd.read_csv(input_path)
tcrb_data = tcrb_data.drop_duplicates()
tcrb_data.head()

In [ ]:
len(tcrb_data)

In [7]:
# rename sequences column if not named "Sequences"
tcrb_data.rename(columns={'sequences': 'Sequences'}, inplace=True)

#### Create embeddings

In [ ]:
from cvc.embbeding_wrapper import EmbeddingWrapper

# Create embeddings
embed_wrap = EmbeddingWrapper(TRANSFORMER, DEVICE, tcrb_data, batch_size=1024, method="mean", layers=[-1])
embed_wrap.embeddings.shape

In [ ]:
tcrb_embeddings_df = pd.DataFrame(embed_wrap.embeddings)
tcrb_embeddings_df

In [12]:
# output embeddings to csv
tcrb_embeddings_df.to_csv(output_path)